# User Interface

### Import libraries

In [9]:
from osgeo import gdal
import tkinter as tk
from tkinter import filedialog, Toplevel, Label
from PIL import Image, ImageTk
import numpy as np
import os
import tkinter as tk
from PIL import Image, ImageTk, ImageEnhance

### Set up the working path

In [10]:
os.chdir('D:\E_2024_P6\SEAL')

## Create the UserInterface

In [11]:
# Global variable to store the selected image
selected_image = None

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("NavySeal")
        self.geometry("800x600")

        # Create a canvas
        self.canvas = tk.Canvas(self, width=800, height=600)
        self.canvas.pack(fill="both", expand=True)

        # Load the background image
        self.bg_image = Image.open("seals.jpg")
        self.bg_image = self.bg_image.resize((800, 600), Image.LANCZOS)

        # Adjust the transparency of the image
        self.bg_image = self.bg_image.convert("RGBA")
        alpha = self.bg_image.split()[3]
        alpha = ImageEnhance.Brightness(alpha).enhance(0.5)  # Adjust transparency to 50%
        self.bg_image.putalpha(alpha)

        # Convert the image with transparency to PhotoImage
        self.bg_photo = ImageTk.PhotoImage(self.bg_image)

        # Add the background image to the canvas
        self.canvas.create_image(0, 0, image=self.bg_photo, anchor="nw")

        # Keep a reference to the image to prevent it from being garbage collected
        self.canvas.image = self.bg_photo

        # Add other widgets on top of the canvas
        self.panel = tk.Label(self)
        self.canvas.create_window(400, 300, window=self.panel)

        self.btn_select = tk.Button(self, text="Select Image", command=self.select_image)
        self.canvas.create_window(400, 450, window=self.btn_select)

        self.btn_process = tk.Button(self, text="Process Data", command=self.process_data)
        self.canvas.create_window(400, 500, window=self.btn_process)

        self.result_label = tk.Label(self, text="File path: ")
        self.canvas.create_window(400, 550, window=self.result_label)

    def read_and_display_tif(self, file_path):
        global selected_image
        try:
            selected_image = Image.open(file_path)
            selected_image = selected_image.resize((500, 500))  # Resize to fit the Tkinter window
            img_tk = ImageTk.PhotoImage(selected_image)
            self.panel.config(image=img_tk)
            self.panel.image = img_tk

            self.result_label.config(text=f"File path: {file_path}")
        except Exception as e:
            print(f"Error reading TIF file: {e}")

    def select_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("TIF Files", "*.tif")])
        if file_path:
            self.read_and_display_tif(file_path)

    def process_data(self):
        global selected_image
        if selected_image is None:
            self.result_label.config(text="No image selected")
            return

        # Display "Please Wait" popup
        please_wait_popup = Toplevel(self)
        please_wait_popup.title("Processing")
        please_wait_label = Label(please_wait_popup, text="Currently in Navy SEALs, please wait.")
        please_wait_label.pack(padx=10, pady=10)

        # Simulate data processing
        self.after(2000, lambda: self.finish_processing(please_wait_popup))

    def finish_processing(self, popup):
        global selected_image
        # Close the popup
        popup.destroy()

        # Simulate processed data (for now just creating an inverted image)
        # Replace this with actual data processing logic
        processed_data = np.invert(np.array(selected_image))  # Example: Invert the selected image
        img = Image.fromarray(processed_data)
        img = img.resize((500, 500))  # Resize to fit the Tkinter window
        img_tk = ImageTk.PhotoImage(img)
        self.panel.config(image=img_tk)
        self.panel.image = img_tk

        self.result_label.config(text="Processing complete")

if __name__ == "__main__":
    app = App()
    app.mainloop()